In [ ]:
# This cell for generating ood dataset
import random
import numpy as np
import os
save_mode = False
def generating_scenarios_dataset(data_dict, train_sc, save_mode=False):
    all_x, all_y, all_test_x, all_test_y = [], [], [], []
    for scenario in data_dict.keys():
        data = data_dict[scenario]
        x_offsets = np.sort(
                # np.concatenate(([-week_size + 1, -day_size + 1], np.arange(-11, 1, 1)))
                np.concatenate((np.arange(-11, 1, 1),))
            )
            # Predict the next one hour
        y_offsets = np.sort(np.arange(1, 13, 1))
        min_t = abs(min(x_offsets))
        max_t = abs(data.shape[0]- abs(max(y_offsets)))

        # max_t = abs(N - abs(max(y_offsets)))  # Exclusive
        x_train, y_train, x_test, y_test = [], [], [], []
        for t in range(min_t, max_t):
            x_t = data[t + x_offsets, ...]
            y_t = data[t + y_offsets, ...]
            if scenario in train_sc:
                x_train.append(x_t)
                y_train.append(y_t)
            # else:
            #     x_test.append(x_t)
            #     y_test.append(y_t)

        # divide train scenerio and test
        if scenario in train_sc:
            x_train = np.stack(x_train, axis=0)
            y_train = np.stack(y_train, axis=0)
            all_x.append(x_train)
            all_y.append(y_train)
        # else:
        #     x_test = np.stack(x_test, axis=0)
        #     y_test = np.stack(y_test, axis=0)
        #     all_test_x.append(x_test)
        #     all_test_y.append(y_test)



    zipped_lists = list(zip(all_x, all_y))
    random.shuffle(zipped_lists)  # shuffle data
    all_x, all_y = zip(*zipped_lists)

    x = np.concatenate(all_x, axis=0)
    y = np.concatenate(all_y, axis=0)



    num_samples, num_nodes = x.shape[0], x.shape[2]  # num_samples = ts - 12*2 +1
    # len_train = round(num_samples * 0.8)
    # len_val = round(num_samples * 0.1)
    x_train, y_train = x, y
    # x_val, y_val = x[len_train:, ...], y[len_train:, ...]
    # x_test, y_test = np.concatenate(all_test_x, axis=0), np.concatenate(all_test_y, axis=0)

    if save_mode:
        for cat in ["train"]:
            _x, _y = locals()["x_" + cat], locals()["y_" + cat]
            np.savez_compressed(
            os.path.join("./ood_dataset", "%s.npz" % cat),
            x=_x,
            y=_y,
            # x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
            # y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
        )

In [ ]:
from lib.utils import gen_data_dict
from lib.utils import process_sensor_data
# Define path to parent directory containing subdirectories with CSV files
parent_dir = './sc sensor 2/'
df_dict = dict()
train_sc = ["sc1", "sc2", "sc2_2", "sc3", "sc3_2",
            "sc5", "sc5_2", "sc13", "sc13_2", "sc16",
            "sc18", "sc19"]
# Loop through each subdirectory in the parent directory
df_dict = process_sensor_data(parent_dir, df_dict)
data_dict = gen_data_dict(df_dict)
generating_scenarios_dataset(data_dict, train_sc)